# Dependencies

In [1]:
import json
import urllib.request
import requests
import pandas as pd
import sqlite3

# API Queries

In [22]:
#Measurements
m_url = 'https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=bfa9f262-24a1-45bd-8dc8-138bc8107266'

# Stations
s_url = 'https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=af157380-fb42-4abf-b72a-6f9f98868077'

# Well Perforations
p_url = 'https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=f1deaa6d-2cb5-4052-a73f-08a69f26b750'

# Get Measurements data

In [23]:
response = requests.get(m_url)

if response.status_code == 200:
    data = response.json()
    records = data.get('result', {}).get('records', [])
    m_df = pd.DataFrame(records)
    print('Successfully retrieved measurement data')
else:
    print(f"Failed to retrieve measurement data: {response.status_code} - {response.text}")

Successfully retrieved measurement data


### In Measurements convert msmt_date to year

In [24]:
m_df['year'] = m_df['msmt_date'].str[:4]
m_df['year'] = m_df['year'].astype(int)

# Get Stations data

In [25]:
response = requests.get(s_url)

if response.status_code == 200:
    data = response.json()
    records = data.get('result', {}).get('records', [])
    s_df = pd.DataFrame(records)
    print('Successfully retrieved station data')
else:
    print(f"Failed to retrieve station data: {response.status_code} - {response.text}")

Successfully retrieved station data


# Get Well Perforations

In [26]:
response = requests.get(p_url)

if response.status_code == 200:
    data = response.json()
    records = data.get('result', {}).get('records', [])
    p_df = pd.DataFrame(records)
    print('Successfully retrieved Perforations data')
else:
    print(f"Failed to retrieve Perforations data: {response.status_code} - {response.text}")

Successfully retrieved Perforations data


# Check length of df's

In [27]:
print(f"Measurements is: {len(m_df)}.  Stations is: {len(s_df)}.  Perforations is: {len(p_df)}")

Measurements is: 100.  Stations is: 100.  Perforations is: 100


# Export to SQLite database

In [28]:
conn = sqlite3.connect('Resources/Groundwater.db')

# Measurements
m_df.to_sql('Measurements', conn, if_exists='replace', index=False)

# Stations
s_df.to_sql('Stations', conn, if_exists='replace', index=False)

# Perforations
p_df.to_sql('Perforations', conn, if_exists='replace', index=False)

conn.close()

# Create View

In [ ]:
conn = sqlite3.connect('Resources/Groundwater.db')

# Create a cursor object
cursor = conn.cursor()

# SQL command to create the view
create_view_sql = """
CREATE VIEW vw_groundwater AS
SELECT 
    M.year,
    M.site_code,
    M.gse_gwe,
    M.gwe,
    M.wlm_gse,
    M.wlm_rpe,
    S.latitude,
    S.longitude,
    S.gse,
    S.rpe,
    S.well_depth,
    P.top_prf_int AS p_top_prf_int,  -- Alias to avoid column name conflict
    P.bot_prf_int
FROM 
    Measurements M
LEFT JOIN 
    Stations S ON M.site_code = S.site_code
LEFT JOIN 
    Perforations P ON M.site_code = P.site_code;
"""

# Execute the SQL command to create the view
cursor.execute(create_view_sql)

# Commit the changes
conn.commit()

# Close the connection
conn.close()

# Query View

In [34]:
conn = sqlite3.connect('Resources/Groundwater.db')
query = "SELECT * FROM vw_groundwater"
df = pd.read_sql_query(query, conn)
conn.close()
df.head(10)

,year,site_code,gse_gwe,gwe,wlm_gse,wlm_rpe,latitude,longitude,gse,rpe,well_depth,p_top_prf_int,bot_prf_int
0,2023,320000N1140000W001,423.1,122.82,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
1,2023,320000N1140000W001,383.5,162.42,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
2,2023,320000N1140000W001,385.0,160.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
3,2023,320000N1140000W001,377.0,168.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
4,2023,320000N1140000W001,384.2,161.72,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
5,2021,320000N1140000W001,412.0,133.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
6,2021,320000N1140000W001,410.0,135.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
7,2021,320000N1140000W001,411.0,134.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
8,2021,320000N1140000W001,411.0,134.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
9,2021,320000N1140000W001,409.0,136.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
